In [3]:
!pip install langchain faiss-cpu pypdf sentence-transformers transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 4.4 MB/s eta 0:00:00


In [4]:
from google.colab import files

uploaded_files = files.upload()  # Upload your 10 PDFs here
pdf_files = list(uploaded_files.keys())


Saving ccr-9501-clark-design-philosophy-of-DARPA-Internet-protocols.pdf to ccr-9501-clark-design-philosophy-of-DARPA-Internet-protocols.pdf
Saving ck74-protocol-for-packet-network-intercommunication.pdf to ck74-protocol-for-packet-network-intercommunication.pdf
Saving endtoend-arguments-in-systems-design.pdf to endtoend-arguments-in-systems-design.pdf
Saving FQ1989-Analysis-and-Simulation-of-Fair-Queuing-Algorithm.pdf to FQ1989-Analysis-and-Simulation-of-Fair-Queuing-Algorithm.pdf
Saving Scalable-High-Speed-IP-routing-lookups.pdf to Scalable-High-Speed-IP-routing-lookups.pdf


In [6]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [14]:
from PyPDF2 import PdfReader

def extract_text_from_pdfs(file_list):
    all_texts = []
    for file in file_list:
        reader = PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        all_texts.append(text)
    return all_texts

documents = extract_text_from_pdfs(pdf_files)



In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = []
for doc in documents:
    chunks += splitter.split_text(doc)


In [19]:
pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [25]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
test_embedding = embedding_model.embed_query("This is a test sentence.")
print("Embedding vector length:", len(test_embedding))

for i in range(15,25,1):
  print("Number of chunks created:", len(chunks))
  print("Example chunk:\n", chunks[i][:300])

vectorstore = FAISS.from_texts(chunks, embedding_model)
print("Done")

Embedding vector length: 384
Number of chunks created: 746
Example chunk:
 The final aspect of this fundamental goal was the
assumption of the particular technique for inter-connecting these networks. Since the technique of storeand forward packet switching, as demonstrated in theprevious DARPA project, the ARPANET, was wellunderstood, the top level assumption was that net
Number of chunks created: 746
Example chunk:
 From these assumptions comes the fundamental
structure of the Internet: a packet switched communica-tions facility in which a number of distinguishablenetworks are connected together using packet communi-cations processors called gateways which implement astore and forward packet forwarding algorith
Number of chunks created: 746
Example chunk:
 3. Second Level Goals
The top level goal stated in the previous sectioncontains the word "effective," without offering anydefinition of what an effective interconnection mustachieve. The following list summarizes a more detailedset

In [27]:
query = "What is this paper about?"
results = vectorstore.similarity_search(query, k=1)
print("Retrieved chunk:\n\n", results[0].page_content[:1000])



Retrieved chunk:

 Introduction


In [29]:
query = "What networking technologies are discussed?"
results = vectorstore.similarity_search(query, k=2)
for i, chunk in enumerate(chunks[:5]):
    print(f"\n--- Chunk {i+1} ---\n{chunk[:500]}")




--- Chunk 1 ---
ACM SIGCOMM -1- Computer Communication ReviewThe Design Philosophy of the DARPA Internet Protocols
David D. Clark*
Massachusetts Institute of Technology
Laboratory for Computer Science
Cambridge, MA. 02139
(Originally published in Proc. SIGCOMM ‘88, Computer Communication Review Vol. 18, No. 4,
August 1988, pp. 106–114)

--- Chunk 2 ---
August 1988, pp. 106–114)
                                                
*This work was supported in part by the Defense Advanced Research Projects Agency (DARPA) under Contract No. N00014-83-K-0125Abstract
The Internet protocol suite, TCP/IP, was first proposed

--- Chunk 3 ---
fifteen years ago. It was developed by the DefenseAdvanced Research Projects Agency (DARPA), andhas been used widely in military and commercialsystems. While there have been papers andspecifications that describe how the protocols work, it issometimes difficult to deduce from these why theprotocol is as it is. For example, the Internet protocol isbased on a co

In [30]:
from transformers import pipeline

llm = pipeline("text2text-generation", model="google/flan-t5-base", max_length=512)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [31]:
def ask_question(query):
    relevant_docs = vectorstore.similarity_search(query, k=3)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    prompt = f"Answer the question based only on the following text:\n\n{context}\n\nQuestion: {query}"
    answer = llm(prompt)
    return answer[0]['generated_text']

# Example:
ask_question("What is the main idea of the third paper?")


'the process by which this project was managed'

In [32]:
while True:
    query = input("\nAsk something about your papers (or type 'exit'): ")
    if query.lower() == 'exit':
        break
    print("\nAnswer:", ask_question(query))



Ask something about your papers (or type 'exit'): are those papers based on networking

Answer: yes

Ask something about your papers (or type 'exit'): what are the topics of these papers

Answer: ad hoc networks

Ask something about your papers (or type 'exit'): whay are you giving short answers

Answer: no


KeyboardInterrupt: Interrupted by user